### multi-classification
: multi-nomial classification (다중 분류) : Y값의 범주가 3개 이상인 분류
#### 활성화 함수(Activation function) 으로 softmax함수 가 사용된다

In [1]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(5)

In [2]:
# train data set :
# x_data :  [N,4]  --> [8,4]
x_data = [[1,2,1,1],
          [2,1,3,2],
          [3,1,3,4],
          [4,1,5,5],
          [1,7,5,5],
          [1,2,5,6],
          [1,6,6,6],
          [1,7,7,7]]

# y_data : [N,3] --> [8,3]
y_data = [[0,0,1],  # [2]
          [0,0,1],  # [2]
          [0,0,1],  # [2]
          [0,1,0],  # [1]
          [0,1,0],  # [1]
          [0,1,0],  # [1]
          [1,0,0],  # [0]
          [1,0,0]]  # [0]

x_train = np.array(x_data,dtype=np.float32)
y_train = np.array(y_data,dtype=np.float32)

In [3]:
nb_classes = 3
# 변수 초기화 : weight, bias
# (m,n) * (n,l) = (m,l)   : 행렬의 내적 곱셈 공식
# (8,4) * (4,3) = (8,3)
W = tf.Variable(tf.random.normal([4,nb_classes]),name='weight')
b = tf.Variable(tf.random.normal([nb_classes]),name='bias')
print(W)
print(b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.18030666, -0.95028627, -0.03964049],
       [-0.7425406 ,  1.3231523 , -0.61854804],
       [ 0.8540664 , -0.08899953,  2.4488697 ],
       [ 0.762508  ,  1.2659615 ,  0.9801489 ]], dtype=float32)>
<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0.22652863, 0.8106553 , 0.7466094 ], dtype=float32)>


In [4]:
# 예측 함수(hypothesis) : H(X) = softmax(X*W + B)
def logits(X):
    return tf.matmul(X,W) + b

def hypothesis(X):
    return tf.nn.softmax(logits(X))  

In [5]:
#  비용 함수 구현 방법 1: log함수를 사용하여 수식을 직접 표현
# def cost_func():
#     cost = tf.reduce_mean(-tf.reduce_sum(y_train*tf.math.log(hypothesis(x_train)),
#                                          axis=1))
#     return cost

In [6]:
#  비용함수 구현 방법 2 : tf.nn.softmax_cross_entropy_with_logits() 함수 사용
def cost_func():
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(x_train),
                                                    labels=y_train)
    cost = tf.reduce_mean(cost_i)
    return cost

In [7]:
# 경사 하강법
# learning_rate(학습율)을 0.01 로 설정하여 optimizer객체를 생성
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [8]:
# 학습 시작
print('***** Start Learning!!')
for step in range(5001):
    optimizer.minimize(cost_func,var_list=[W,b])
    if step % 1000 == 0:
        print('%04d'%step,'cost:[',cost_func().numpy(),']',
             ' W:',W.numpy(),' b:',b.numpy())
print('***** Learning Finished!!') 

***** Start Learning!!
0000 cost:[ 5.9294786 ]  W: [[-0.17030679 -0.94028634 -0.04964045]
 [-0.7325406   1.3331522  -0.628548  ]
 [ 0.86406636 -0.07899956  2.4388697 ]
 [ 0.77250797  1.2759615   0.9701489 ]]  b: [0.2365285  0.8206551  0.73660946]
1000 cost:[ 0.31467548 ]  W: [[-2.21942592e+00  5.24806201e-01  1.18725026e+00]
 [ 1.21048994e-01  2.25326256e-03 -6.37006015e-02]
 [ 2.28707361e+00  1.16509998e+00  6.74349904e-01]
 [ 1.27608597e+00  1.77689016e+00  3.76738161e-01]]  b: [-2.7187693 -1.5067683  4.317265 ]
2000 cost:[ 0.16304705 ]  W: [[-3.7972453   1.3537484   2.2676597 ]
 [ 0.04149103  0.02373767  0.06498571]
 [ 3.5885108   1.0490662  -0.5021981 ]
 [ 1.0556884   1.8170305   0.5562784 ]]  b: [-6.115444  -1.3375343  7.16916  ]
3000 cost:[ 0.08965954 ]  W: [[-5.2343025   2.166253    3.208689  ]
 [-0.04391025  0.06965824  0.1601942 ]
 [ 4.872835    0.69199187 -1.5755049 ]
 [ 0.73721075  2.0572217   0.7426433 ]]  b: [-9.075818  -1.3604151  9.7624235]
4000 cost:[ 0.051186893 ]  W: 

In [9]:
# weight과 bias 출력
print('Weight:',W.numpy())
print('Bias:',b.numpy())

Weight: [[-7.8441839e+00  3.6925719e+00  4.8953743e+00]
 [-2.1366459e-01  1.7167655e-01  3.2927501e-01]
 [ 7.3285408e+00 -1.4589469e-01 -3.5808372e+00]
 [-2.5778222e-03  2.6041069e+00  1.1803559e+00]]
Bias: [-14.224036   -1.3462137  14.310632 ]


In [14]:
# 예측
# tf.argmax() : 값이 가장 큰 요소의 인덱스 값을 반환
def predict(X):
    return tf.argmax(hypothesis(X),axis=1)

# 학습 데이터를 검증 데이터로 동일하게 사용하는 경우
x_test = x_train
y_test = y_train

preds = predict(x_test)
print(preds.numpy())
print(hypothesis(x_test).numpy())
print(tf.argmax(y_test,axis=1).numpy())

[2 2 2 1 1 1 0 0]
[[6.7201633e-15 4.4787926e-06 9.9999547e-01]
 [3.0885534e-11 6.2573748e-03 9.9374264e-01]
 [8.2858872e-18 3.1583689e-02 9.6841633e-01]
 [5.7242741e-16 9.7510344e-01 2.4896599e-02]
 [5.8690630e-02 9.3963599e-01 1.6734080e-03]
 [3.0667992e-02 9.6914297e-01 1.8900885e-04]
 [9.2271829e-01 7.7280879e-02 9.1235961e-07]
 [9.9905401e-01 9.4603369e-04 1.0147081e-10]]
[2 2 2 1 1 1 0 0]
